In [11]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [12]:
# Loading Libraries
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from torch import optim
from torch.autograd import Variable
from torchvision.transforms import Compose
from torchvision.datasets import CIFAR10

In [13]:
device = "cpu"

if torch.cuda.is_available():
    device = "cuda"

print(device)

cuda


In [14]:
train_transform = Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transform = Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

cifar10_train = CIFAR10(root = "/data", train=True, download = True, transform=train_transform)
train_loader = torch.utils.data.DataLoader(cifar10_train, batch_size=64, shuffle=True)

cifar10_test = CIFAR10(root = "/data", train=False, download = True, transform=test_transform)
test_loader = torch.utils.data.DataLoader(cifar10_test, batch_size=64, shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


In [15]:
vgg = models.vgg16(pretrained=True)
vgg = vgg.to(device)

In [16]:
is_cuda = False
if torch.cuda.is_available():
    is_cuda = True

In [17]:
vgg.classifier[6].out_features = 10
for param in vgg.features.parameters(): param.requires_grad = False

In [18]:
num_epochs = 5
num_classes = 10
learning_rate = 0.001

In [19]:
optimizer = optim.SGD(vgg.classifier.parameters(),lr=0.005,momentum=0.5)
criterion = nn.CrossEntropyLoss()

In [20]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = vgg(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))


Epoch [1/5], Step [100/782], Loss: 1.9152
Epoch [1/5], Step [200/782], Loss: 1.2515
Epoch [1/5], Step [300/782], Loss: 1.4755
Epoch [1/5], Step [400/782], Loss: 1.3484
Epoch [1/5], Step [500/782], Loss: 1.5471
Epoch [1/5], Step [600/782], Loss: 1.3955
Epoch [1/5], Step [700/782], Loss: 1.1922
Epoch [2/5], Step [100/782], Loss: 1.4971
Epoch [2/5], Step [200/782], Loss: 1.3904
Epoch [2/5], Step [300/782], Loss: 1.2556
Epoch [2/5], Step [400/782], Loss: 1.3125
Epoch [2/5], Step [500/782], Loss: 1.4636
Epoch [2/5], Step [600/782], Loss: 1.4264
Epoch [2/5], Step [700/782], Loss: 1.2851
Epoch [3/5], Step [100/782], Loss: 1.3450
Epoch [3/5], Step [200/782], Loss: 1.3611
Epoch [3/5], Step [300/782], Loss: 1.4913
Epoch [3/5], Step [400/782], Loss: 1.0109
Epoch [3/5], Step [500/782], Loss: 1.1550
Epoch [3/5], Step [600/782], Loss: 1.4212
Epoch [3/5], Step [700/782], Loss: 1.3468
Epoch [4/5], Step [100/782], Loss: 1.2259
Epoch [4/5], Step [200/782], Loss: 1.5160
Epoch [4/5], Step [300/782], Loss:

In [21]:
# Test the model
vgg.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = vgg(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))


Test Accuracy of the model on the 10000 test images: 60.9 %
